# This notebook is still work in progress!

This notebook has the following dependencies. Make sure that these are installed in your environment when you launch this notebook.
+ pandas
+ biopython
+ numpy
+ openpyxl

For more information refer to the [website](https://gitlab.com/NCDRlab/easy_hcr)

In [1]:
import pandas as pd
import numpy as np
import csv
from insitu_probe_generator.maker37cb import maker
from insitu_probe_generator.start import start
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIXML

In [2]:
# Set the quality control parameters for the probeset
remove_gc = False
probepair_number_cutoff = 33
blast_on_genome = True

In [ ]:
strt = start()
name,fullseq,amplifier,pause,choose,polyAT,polyCG,BlastProbes,db,dropout,show,report,maxprobe,numbr = strt[0],strt[1],strt[2],strt[3],strt[4],strt[5],strt[6],strt[7],strt[8],strt[9],strt[10],strt[11],strt[12],strt[13]
string_output = maker(name,fullseq,amplifier,pause,choose,polyAT,polyCG,BlastProbes,db,dropout,show,report,maxprobe,numbr)

In [ ]:
# Get and re-usethe parameters specified for the probe generator

gene_name = name
amplifier = amplifier

print("Gene: ", gene_name)
print("Amplifier: ", amplifier)

In [ ]:
# Save the generated probes as CSV file

csv_path = f"output/{name}_probes.csv"

probe_file = open(csv_path, "w")
n = probe_file.write(string_output)
probe_file.close()

# Read the csv back in as table

df = pd.read_csv(
    f"output/{name}_probes.csv",
    index_col=0,
)
print("Gene: ", gene_name)
print("Amplifier: ", amplifier)
print("Probes:")
df.head()

In [ ]:
# Generate probe identifiers

probe_identifiers = [gene_name + "_PP_" + str(n + 1) for n,_ in enumerate(df.iterrows())]
probe_identifiers = [f"{gene_name}_{amplifier}_PP_{str(n + 1)}" for n,_ in enumerate(df.iterrows())]
df.index = probe_identifiers
df.head()

In [ ]:
# Check GC-ending probes for later
# GC and the end for the forst probe and GC at the start for the second

df["GC_QC"] = df["Probe"].str.endswith("GC") | df["Probe"].str.endswith("CG") | df["Probe.1"].str.startswith("GC") | df["Probe.1"].str.startswith("CG")
print("Number of probe pairs:", len(df))
print(f"{len(df.loc[df.GC_QC == True])} GC probe pairs")
print(f"Kept {len(df.loc[df.GC_QC != True])} no-GC probe pairs")
df


In [8]:
# Generate BLAST sequences

df["blast"] = df["Probe"] + "XX" + df["Probe.1"]
df.head()

,Initiator,Spacer,Probe,Probe.1,Spacer.1,Initiator.1,GC_QC,blast
Lyve1_B1_PP_1,GAGGAGGGCAGCAAACGG,aa,GCGGCAGCACCAAAGAAGAGGAGAG,CTTTTCACGTAGCAAACAGCCAGCA,ta,GAAGAGTCTTCCTTTACG,False,GCGGCAGCACCAAAGAAGAGGAGAGXXCTTTTCACGTAGCAAACAG...
Lyve1_B1_PP_2,GAGGAGGGCAGCAAACGG,aa,CAAACCCAGCTGCTTCGTTCTTGAA,GCACCAGCAGGGCGGTGGGGACACC,ta,GAAGAGTCTTCCTTTACG,True,CAAACCCAGCTGCTTCGTTCTTGAAXXGCACCAGCAGGGCGGTGGG...
Lyve1_B1_PP_3,GAGGAGGGCAGCAAACGG,aa,TTCTGTAGCCATGGTGATAGGTTCT,TGCTCCACTTGCAACAAATGCTTCT,ta,GAAGAGTCTTCCTTTACG,False,TTCTGTAGCCATGGTGATAGGTTCTXXTGCTCCACTTGCAACAAAT...
Lyve1_B1_PP_4,GAGGAGGGCAGCAAACGG,aa,AAGGTGGAGCCCGGGTGGTGGCAGA,TTTTTGTCTTCCGTGCCATGGAGGT,ta,GAAGAGTCTTCCTTTACG,False,AAGGTGGAGCCCGGGTGGTGGCAGAXXTTTTTGTCTTCCGTGCCAT...
Lyve1_B1_PP_5,GAGGAGGGCAGCAAACGG,aa,CAAGTAGGCGCTGCTGCTGACAGAA,AGGTGTTGTGGAGTCAGGGGATGAA,ta,GAAGAGTCTTCCTTTACG,False,CAAGTAGGCGCTGCTGCTGACAGAAXXAGGTGTTGTGGAGTCAGGG...


In [ ]:
# Create a simple dataframes for BLAST

df_blast = pd.DataFrame(df.index.values, columns=["Identifier"])
df_blast["Probe"] = df.blast.values
df_blast.to_csv(f"output/{gene_name}_probes.txt",sep='\t',index=False,header=False, quoting=csv.QUOTE_NONE)
df_blast.head()

In [ ]:
# Generate a list of SeqRecord to write a fasta file

probe_sequences = [SeqRecord(Seq(probe), id=id) for probe, id in zip(df_blast.Probe.values, df_blast.Identifier.values)]

# Print first 5 as example
probe_sequences[:5]

In [12]:
# Save fast file in output folder

fasta_path = f"output/{gene_name}_to_blast.fasta"

with open(fasta_path, "w") as output_handle:
    for record in probe_sequences:
        SeqIO.write(record, output_handle, "fasta")

In [13]:
# Run BLAST (still to implement)

# from Bio.Blast import NCBIWWW
# fasta_string = open(fasta_path).read()
# result_handle = NCBIWWW.qblast("blastn", "refseq_rna", fasta_string)

In [14]:
# Path of the BLAST output

blast_path = f"output/{gene_name}_blast_output.xml"

In [16]:
result_handle = open(blast_path)

In [17]:
blast_records = list(NCBIXML.parse(result_handle))
result_handle.close()

In [ ]:
# Show first entry as example

for alignment in blast_records[0].alignments:
    print(alignment)

In [19]:
# #Show all entries
# for record in blast_records:
#     for alignment in record.alignments:
#         print(alignment)

In [ ]:
# Generate a table of results

queries = []
names = []
types = []

for record in blast_records:
    for alignment in record.alignments:
        queries.append(record.query)
        title = alignment.title
        _, name = title.split("| ")
        name_parts = name.split(", ")
        hit_type = name_parts[-1].split(".")[0]
        hit_type = "DNA" if hit_type != "mRNA" else "mRNA"
        types.append(hit_type)
        name = ", ".join(name_parts[0:-1])
        names.append(name)
        # print(name, hit_type)
    

df_blast_results = pd.DataFrame(queries, columns=["Probe"])
df_blast_results["Name"] = names
df_blast_results["Type"] = types
df_blast_results.head()

In [21]:
# Check unique RNA hits

# TODO
#   Filter out predicted (maybe no) > label it
#   Filter out other genes
#   Not filter correct gene + predicted

print("List of mRNA hits:\n")
type_counts = df_blast_results.loc[df_blast_results.Type == "mRNA"].Name.value_counts()
print(type_counts)
max_count = type_counts.argmax()
true_hit_rna = type_counts.index.values[max_count]
print("\nTrue hit:\n", true_hit_rna)

List of mRNA hits:

Mus musculus lymphatic vessel endothelial hyaluronan receptor 1 (Lyve1)    15
Name: Name, dtype: int64

True hit:
 Mus musculus lymphatic vessel endothelial hyaluronan receptor 1 (Lyve1)


In [ ]:
# Display off-topic predicted binding

other_genes = [name for i, name in enumerate(type_counts.index) if gene_name not in name]
other_genes

In [ ]:
probes_to_remove_rna = []

for name, row in df_blast_results.iterrows():
    if row.Name in other_genes:
        probes_to_remove_rna.append(row.Probe)

probes_to_remove_rna = pd.Series(probes_to_remove_rna).unique()
probes_to_remove_rna

In [ ]:
# Check unique DNA hits

if blast_on_genome:
    print("List of DNA hits:\n")
    name_counts = df_blast_results.loc[df_blast_results.Type == "DNA"].Name.value_counts()
    print(name_counts)
    max_count = name_counts.argmax()
    true_hit_dna = name_counts.index.values[max_count]
    print("\nTrue hit: ", true_hit_dna)

In [ ]:
# List of hits to remove

if blast_on_genome:
    print("Hits to remove")
    df_dna_outliers = df_blast_results.loc[df_blast_results.Type == "DNA"].loc[df_blast_results.Name != true_hit_dna]
    probes_to_remove_dna = df_dna_outliers.Probe.unique()
    df_dna_outliers

In [ ]:
# List of probes to remove

if blast_on_genome:
    probes_to_remove = pd.Series(probes_to_remove_rna.tolist() + probes_to_remove_dna.tolist()).unique()
else:
    probes_to_remove = pd.Series(probes_to_remove_rna.tolist()).unique()
probes_to_remove

In [ ]:
# Remove unwanted probes

on_topic_indexes = [n for n, probe in enumerate(df.index.values) if probe not in probes_to_remove]
df_qc_blast = df.iloc[on_topic_indexes]
print(f"{len(on_topic_indexes)} probes kept:\n", df_qc_blast.index.unique().values)

df_qc_blast.head()

In [28]:
## OPTIONAL
# Remove probes ending in GC or CG

if remove_gc == True:
    orig_number = len(df_qc_blast)
    print("Number of probe pairs:", orig_number)
    df_qc_blast = df_qc_blast.loc[df_qc_blast.GC_QC != True]
    print(f"Removed {orig_number - len(df_qc_blast)} probe pairs")
    print(f"Preserved {len(df_qc_blast)} no-GC probe pairs")
    df_qc_blast

In [ ]:
# If there are more than 50 probe pairs, keep only the odd rows, otherwise cut off at the specified amount

n_probes = len(df_qc_blast)
print("Total number of probes: ", n_probes)

if len(df_qc_blast.iloc[::2]) > 25:
    df_qc_blast = df_qc_blast.iloc[::2]
    print(f"Kept odd rows ({len(df_qc_blast)} probes)")

df_qc_blast = df_qc_blast.iloc[:probepair_number_cutoff]

new_length = len(df_qc_blast)
n_probes = new_length
print("Number of kept probes: ", n_probes)

In [ ]:
# Generate IDT order form

df_idt = pd.DataFrame(columns=["Pool name", "Sequence"])
for name, row in df_qc_blast.iterrows():
    name_parts = row.name.split("_")
    df_idt = df_idt.append({
        "Pool name": "_".join(name_parts[:-2]) + f"_{n_probes}PP",
        "Sequence": row.Initiator + row.Spacer.upper() + row.Probe
    },
    ignore_index=True)
    df_idt = df_idt.append({
        "Pool name": "_".join(name_parts[:-2]) + f"_{n_probes}PP",
        "Sequence": row["Probe.1"] + row["Spacer.1"].upper() + row["Initiator.1"]
    },
    ignore_index=True)
print(f"Generated an order of {len(df_qc_blast)} probe pairs ({len(df_idt)} oligos)")
df_idt

In [31]:
# Export order form

idt_path = f"output/{gene_name}_idt_order.xlsx"

df_idt.to_excel(idt_path, index=False)